# modeling14-3-FiD-encoder-sentence-level-classifier-saving-mean-pooling-embedding
- modeling sentence-classifier
- sentence-bert

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [ ]:
from pprint import pprint
from util import utils

In [ ]:
import json
import math
import os
import logging
import sys
import evaluate
from util import utils
from tqdm.auto import tqdm
import pickle

import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification, 
    AutoModel, 
    AutoConfig, 
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
    get_scheduler,
)
from util.arguments import ModelArguments, DataTrainingArguments 
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from FiD.src.model import FiDT5
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from src.data import BinaryCustomDatasetShuffle

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

## Get FiD Encoder Embedding

In [ ]:
testing_file = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/sequence_exclude_no_answer_partial_decisive/sequence_exclude_no_answer_partial_decisive_ctx100id_split_train_1.json'



In [ ]:
testing_data = utils.open_json(testing_file)

In [ ]:
len(testing_data)

In [ ]:
testing_data[0].keys()

In [ ]:
def prepare_sequential_decisive_data(data):
    output = []
    for instance in data:
        question_ = instance['question']
        decisive_ctxs_ = instance['decisive_ctxs']
        decisive_em_pattern_ = instance['decisive_em_pattern']
        id_ = instance['id']

        ctx_lst = []
        for decisive_ctxs in decisive_ctxs_:
            input_ = 'question: ' + question_ + ', '\
                     ' title: ' + decisive_ctxs['title'] + ', '\
                     ' context : ' + decisive_ctxs['text']
            ctx_lst.append(input_)

        template = {
            'id' : id_,
            'em_pattern' : decisive_em_pattern_,
            'ctx' : ctx_lst

        }
        output.append(template)
    return output

In [ ]:
def prepare_sequential_data(data):
    output = []
    for instance in data:
        question_ = instance['question']
        ctxs_ = instance['ctxs']
        em_pattern_ = instance['em_pattern']
        id_ = instance['id']

        ctx_lst = []
        for context in ctxs_:
            input_ = 'question: ' + question_ + ', '\
                     ' title: ' + context['title'] + ', '\
                     ' context : ' + context['text']
            ctx_lst.append(input_)

        template = {
            'id' : id_,
            'em_pattern' : em_pattern_,
            'ctx' : ctx_lst

        }
        output.append(template)
    return output

In [ ]:
if 'decisive' in testing_file:
    print('decisive')
    seq_train_data = utils.prepare_sequential_decisive_data(testing_data)
else:
    print('not decisive')
    seq_train_data = utils.prepare_sequential_data(testing_data)

In [ ]:
# seq_train_data = prepare_sequential_data(testing_data)

In [ ]:
num_labels = 2
model_name_or_path = '/data/philhoon-relevance/FiD/pretrained_models/nq_reader_large'
max_seq_length = 200

In [ ]:
config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
embedding_model = FiDT5.from_pretrained(model_name_or_path)
fid_encoder = embedding_model.encoder.encoder
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [ ]:
len(seq_train_data)

======================== Testing

In [ ]:
# print(seq_train_data[1].keys())
# print(len(seq_train_data[1]['ctx']))

In [ ]:
output = tokenizer(
            seq_train_data[0]['ctx'],
            return_tensors="pt", 
            padding=True,
            truncation=True,
            add_special_tokens=True,
            max_length=max_seq_length)

In [ ]:
print(output['input_ids'].shape)
print(output['attention_mask'].shape)

In [ ]:
embedding = fid_encoder(output['input_ids'], output['attention_mask'])

In [ ]:
print(embedding.last_hidden_state.shape)

In [ ]:
attention_mask = output['attention_mask']

In [ ]:
attention_mask[0]

In [ ]:
token_embeddings = embedding.last_hidden_state
print(token_embeddings.shape)
print(attention_mask.shape)
print(attention_mask.unsqueeze(-1).shape)

In [ ]:
input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

In [ ]:
print(input_mask_expanded.shape)

In [ ]:
sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

In [ ]:
sum_embeddings.shape

In [ ]:
sum_mask = input_mask_expanded.sum(1)

In [ ]:
sum_mask.shape

In [ ]:
sum_mask = torch.clamp(sum_mask, min=1e-9)

In [ ]:
sum_mask.shape

In [ ]:
mean_pooled = pooled = sum_embeddings / sum_mask

In [ ]:
mean_pooled.shape

In [ ]:
input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
sum_mask = input_mask_expanded.sum(1)
sum_mask = torch.clamp(sum_mask, min=1e-9)
pooled = sum_embeddings / sum_mask

======================== Testing End

In [ ]:
# mean_pooled.shape

In [ ]:
fid_encoder.to(device = device)

In [ ]:
fid_encoder.eval()
new_instance = []
for instance in tqdm(seq_train_data):
    input_ = tokenizer(instance['ctx'],
                       return_tensors="pt",
                       padding=True,
                       truncation=True,
                       add_special_tokens=True,
                       max_length=max_seq_length)
    
    input_ids = input_['input_ids'].to(device)
    attention_mask = input_['attention_mask'].to(device)
    
    with torch.no_grad():
        embedding = fid_encoder(input_ids, attention_mask)
        token_embeddings = embedding.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.shape).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_pooled = sum_embeddings / sum_mask

    mean_pooled_cpu = mean_pooled.detach().cpu()
    em_pattern_ = torch.tensor([int(i) for i in instance['em_pattern']])

    result = {
        'input_embedding': mean_pooled_cpu,
        'em_pattern': em_pattern_
    }
    new_instance.append(result)
    

In [ ]:
i = 20

In [ ]:
print(new_instance[i]['input_embedding'].shape)
print(new_instance[i]['em_pattern'].shape)

In [ ]:
import pickle

In [ ]:
output_file = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/sequence_exclude_no_answer_exclude_indecisve/testing-sequence_exclude_no_answer_exclude_indecisve_ctx100id_split_train_1.pickle'


In [ ]:
with open(output_file, 'wb') as f:
    pickle.dump(new_instance, f)

In [ ]:
new_instance[0]

In [ ]:
new_instance[0]

In [ ]:
class EncoderSentenceClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, instances, model, tokenizer, shuffle=True):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.model = model

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = self.tokenizer(self.instances[idx]['ctx'], 
                               return_tensors="pt", 
                               padding=True,
                               truncation=True,
                               add_special_tokens=True,
                               max_length=max_seq_length)
        attention_mask = input_['attention_mask']
        embedding = self.model(input_['input_ids'], attention_mask)
        mean_pooled = embedding.last_hidden_state.sum(axis = -1) / attention_mask.sum(axis=-1).unsqueeze(-1)
        
        em_pattern_ = torch.tensor([int(i) for i in self.instances[idx]['em_pattern']])

        result = {
            'input_embedding': mean_pooled,
            'em_pattern': em_pattern_
        }

        return result

In [ ]:
testing_dataset = EncoderSentenceClassificationDataset(seq_train_data, fid_encoder, tokenizer)

In [ ]:
i = 5

In [ ]:
print(len(seq_train_data[i]['ctx']))

In [ ]:
test_data = testing_dataset[i]

In [ ]:
print(test_data['input_embedding'].shape)
print(test_data['em_pattern'].shape)

In [ ]:
## BinaryCustomDatasetShuffle
#         output = self.tokenizer(
#             input_,
#             # return_tensors="pt", will be applied later through collator
#             # padding=True, will be padded later through collate
#             truncation=True,
#             add_special_tokens=True,
#             max_length=self.max_length)

testing_dataset = BinaryCustomDatasetShuffle(testing_data, tokenizer=tokenizer,
                                 max_length=max_seq_length, shuffle=False)


In [ ]:
print(testing_dataset[0])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

In [ ]:
testing_dataloader = DataLoader(testing_dataset,
                                  shuffle=False,
                                  collate_fn=data_collator,
                                  batch_size=2,
                                  )

In [ ]:
b = iter(testing_dataloader)

In [ ]:
testing_input = next(b)

In [ ]:
# print(testing_input)

In [ ]:
num_labels = 2
model_name_or_path = '/data/philhoon-relevance/FiD/pretrained_models/nq_reader_large'

In [ ]:
config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
model_class = FiDT5
embedding_model = model_class.from_pretrained(model_name_or_path)
fid_encoder = embedding_model.encoder.encoder

In [ ]:
output = fid_encoder(testing_input['input_ids'], testing_input['attention_mask'])

In [ ]:
type(output)

In [ ]:
vars(output)

In [ ]:
output['last_hidden_state'].shape

====================================================================================================

In [ ]:
pprint(vars(tokenizer))

In [ ]:
output = tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=max_seq_length)

In [ ]:
# sentences = ['This framework generates embeddings for each input sentence',
#     'Sentences are passed as a list of string.',
#     'The quick brown fox jumps over the lazy dog.']

In [ ]:
# embeddings = sentence_model.encode(sentences, convert_to_tensor=True)

In [ ]:
# for sentence, embedding in zip(sentences, embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding.shape)
#     print("")


In [ ]:
train_test = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/sequence_exclude_no_answer_exclude_indecisve/sequence_exclude_no_answer_exclude_indecisve_ctx100id_split_train_1.json'



In [ ]:
train_data = utils.open_json(train_test)

In [ ]:
len(train_data)

In [ ]:
# train_data[0].keys()

In [ ]:
def prepare_sequential_data(data):
    output = []
    for instance in data:
        question_ = instance['question']
        ctxs_ = instance['ctxs']
        em_pattern_ = instance['em_pattern']
        id_ = instance['id']

        ctx_lst = []
        for context in ctxs_:
            input_ = 'question: ' + question_ + ', '\
                     ' title: ' + context['title'] + ', '\
                     ' context : ' + context['text']
            ctx_lst.append(input_)

        template = {
            'id' : id_,
            'em_pattern' : em_pattern_,
            'ctx' : ctx_lst

        }
        output.append(template)
    return output

In [ ]:
seq_train_data = prepare_sequential_data(train_data)

In [ ]:
len(seq_train_data)

In [ ]:
print(type(seq_train_data))

In [ ]:
# len(seq_train_data)

In [ ]:
# pprint(vars(embedding_model))
# max_seq

In [ ]:
# 'word_embedding_dimension': 384

In [ ]:
# vars(embedding_model)

In [ ]:
num_labels = 2

In [ ]:
model_name_or_path = '/data/philhoon-relevance/FiD/pretrained_models/nq_reader_large'

In [ ]:
config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [ ]:
model_class = FiDT5

In [ ]:
embedding_model = model_class.from_pretrained(model_name_or_path)

In [ ]:
fid_encoder = embedding_model.encoder.encoder

In [ ]:
inputs = tokenizer(['hi'])

In [ ]:
# print(fid_encoder)

In [ ]:
# tokenizer.convert_ids_to_tokens(inputs['input_ids'], batch=True)

In [ ]:
seq_train_data[0]['em_pattern']

In [ ]:
input_ = seq_train_data[0]['ctx']

In [ ]:
# pprint(input_)

In [ ]:
max_seq_length = 200

In [ ]:
input_[35]

In [ ]:
output = tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=max_seq_length)

In [ ]:
output.keys(

In [ ]:
item = {key: val for key, val in output.items()}

In [ ]:
# pprint(item)

In [ ]:
print(len(output['input_ids']))
print(len(output['attention_mask']))

In [ ]:
type(output['input_ids'][0])

In [ ]:
tokenizer._pad_token_type_id

In [ ]:
# pprint(vars(tokenizer.))

In [ ]:
# embedding_model(**item)

In [ ]:
class EncoderSentenceClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, instances, model, tokenizer,shuffle=True):
        if shuffle:
            random.shuffle(instances)
        self.tokenizer = tokenizer
        self.instances = instances
        self.model = model

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        output = self.tokenizer(self.instances[idx]['ctx'], 
                                truncation=True, 
                                padding=True,
                                add_special_tokens=True, 
                                max_length = self.max_seq_length)
        
#         input_ = torch.from_numpy(self.model.encode(self.instances[idx]['ctx'], show_progress_bar=False))
#         em_pattern_ = torch.tensor([int(i) for i in self.instances[idx]['em_pattern']])

        result = {
            'input_embedding': input_,
            'em_pattern': em_pattern_
        }

        return result

In [ ]:
class BinaryCustomDatasetPredictionShuffle(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = 'question: ' + self.instances[idx]['question'] + ', '\
                 ' title: ' + self.instances[idx]['ctx']['title'] + ', '\
                 ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        # item['labels'] = torch.tensor(int(self.instances[idx]['em']))
        item['labels'] = torch.tensor(int(self.instances[idx]['binary_inference']))

        return item

In [ ]:
class SentenceClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, instances, model, shuffle=True):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.model = model

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = torch.from_numpy(self.model.encode(self.instances[idx]['ctx'], show_progress_bar=False))
        em_pattern_ = torch.tensor([int(i) for i in self.instances[idx]['em_pattern']])

        result = {
            'input_embedding': input_,
            'em_pattern': em_pattern_
        }

        return result

In [ ]:
pprint(config)

In [ ]:
# class SentenceClassificationDataset(torch.utils.data.Dataset):
#     def __init__(self, instances, model, shuffle = True):
#         if shuffle:
#             random.shuffle(instances)
#         self.instances = instances
#         self.model = model

#     def __len__(self):
#         return len(self.instances)

#     def __getitem__(self, idx):
#         input_ = torch.from_numpy(self.model.encode(self.instances[idx]['ctx'], show_progress_bar=False))
#         em_pattern_ = torch.tensor([int(i) for i in self.instances[idx]['em_pattern']])
        
#         result = {
#             'input_embedding' : input_,
#             'em_pattern' : em_pattern_
#         }

#         return result
    
class SentenceClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, instances, model, shuffle=True):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.model = model

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = torch.from_numpy(self.model.encode(self.instances[idx]['ctx'], show_progress_bar=False))
        em_pattern_ = torch.tensor([int(i) for i in self.instances[idx]['em_pattern']])

        result = {
            'input_embedding': input_,
            'em_pattern': em_pattern_
        }

        return result

In [ ]:
train_dataset = SentenceClassificationDataset(seq_train_data, embedding_model)

In [ ]:
# train_dataset[0]

In [ ]:
# def sentence_padding(embedding, cur_max_seq, device):
#     if embedding.shape[0] < cur_max_seq:
#         zero_pad = torch.full(size=(cur_max_seq-embedding.shape[0], embedding.shape[1]), fill_value = -100).to(device=device)
#         print(f'zero_pad.shape : {zero_pad.shape}')
#         return torch.concat([embedding, zero_pad]).to(device=device)
#     else:
#         return embedding[:cur_max_seq, :]

In [ ]:
# input_embedding.shape[0]

In [ ]:
# batch_size = 5

In [ ]:
# # get max number of sentence in a batch
# train_lst = []
# label_lst = []
# max_seq_lst = []
# for i in range(0, batch_size):
#     train_lst.append(train_dataset[i]['input_embedding'])
#     label = train_dataset[i]['em_pattern']
#     label_lst.append(label)
#     max_seq_lst.append(label.shape[0])
# max_seq_len = max(max_seq_lst)

In [ ]:
# train_lst = [d['input_embedding'] for d in batch]
# label_lst = [d['em_pattern'] for d in batch]
# max_seq_lst = max([d['em_pattern'].shape[0] for d in batch])

In [ ]:
# print(max_seq_lst)
# print(max_seq_len)

In [ ]:
# for i in train_lst:
#     print(type(i))
#     print(i.shape)

In [ ]:
# padding_train_lst = []
# for embedding in train_lst:
#     if embedding.shape[0] < max_seq_len:
#         post_pad = torch.full(size=(max_seq_len-embedding.shape[0], embedding.shape[1]), fill_value = -100)
#         padding_train_lst.append(torch.concat([embedding, post_pad]))
#     else:
#         padding_train_lst.append(embedding)

In [ ]:
# input_ = torch.stack(padding_train_lst)

In [ ]:
# input_.shape

In [ ]:
# input_[3,-1,:]

In [ ]:
# print(padding_train_lst[3].shape)
# print(padding_train_lst[3][-2])

In [ ]:
# padding_label_lst = []
# for label in label_lst:
#     if label.shape[0] < max_seq_len:
#         post_pad = torch.full(size=(max_seq_len-label.shape[0], ), fill_value = -100)
#         torch.concat([label, post_pad])
#         padding_label_lst.append(torch.concat([label, post_pad]))
#     else:
#         padding_label_lst.append(label)

In [ ]:
# print(padding_label_lst[4].shape)
# print(padding_label_lst[3])

In [ ]:
# label_ = torch.stack(padding_label_lst)

In [ ]:
# label_.shape

In [ ]:
# label_[3]

In [ ]:
def custom_collate(batch):
    train_lst = [b['input_embedding'] for b in batch]
    label_lst = [b['em_pattern'] for b in batch]
    seq_len_lst = [b['em_pattern'].shape[0] for b in batch]
    max_seq_len = max(seq_len_lst)
    
    padding_train_lst = []
    for embedding in train_lst:
        if embedding.shape[0] < max_seq_len:
            post_pad = torch.full(size=(max_seq_len-embedding.shape[0], embedding.shape[1]), fill_value = -100)
            padding_train_lst.append(torch.concat([embedding, post_pad]))
        else:
            padding_train_lst.append(embedding)
            
    inputs = torch.stack(padding_train_lst)
    
    padding_label_lst = []
    for label in label_lst:
        if label.shape[0] < max_seq_len:
            post_pad = torch.full(size=(max_seq_len-label.shape[0], ), fill_value = -100)
            torch.concat([label, post_pad])
            padding_label_lst.append(torch.concat([label, post_pad]))
        else:
            padding_label_lst.append(label)
            
    labels = torch.stack(padding_label_lst)
    
    return {
        'inputs' : inputs,
        'labels' : labels,
        'sequence_len' : torch.tensor(seq_len_lst)
    }

In [ ]:
batch_size = 128

In [ ]:
train_dataloader = DataLoader(train_dataset,
                                  shuffle=False,
                                  collate_fn=custom_collate,
                                  batch_size=batch_size,
                                  )

In [ ]:
# train_iter = iter(train_dataloader)

In [ ]:
# batch1 = next(train_iter)
# print(batch1['inputs'].shape)
# print(batch1['labels'].shape)
# print(batch1['sequence_len'])

In [ ]:
# x = batch1['inputs']
# y = batch1['labels']
# seq_len = batch1['sequence_len']
# print(seq_len)

In [ ]:
# print(x.shape)
# print(y.shape)
# print(seq_len.shape)


In [ ]:
# print(batch1['labels'])

In [ ]:
# batch1['inputs'][3][-10]

# Model

In [ ]:
class SentenceLSTM(nn.Module):
    def __init__(self, num_layers, embedding_size, num_labels, drop_out_rate):
        super(SentenceLSTM, self).__init__()
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        self.num_labels = num_labels
        self.drop_out_rate = drop_out_rate

        self.lstm = nn.LSTM(input_size=self.embedding_size,
                            hidden_size=self.embedding_size,
                            num_layers=self.num_layers, batch_first=True)

        # Classifier Layers
        self.dropout = nn.Dropout(self.drop_out_rate)
        self.dense = nn.Linear(self.embedding_size, self.embedding_size)
        self.out_proj = nn.Linear(self.embedding_size, self.num_labels)

        # Initializing layers
        self.out_proj.apply(self._init_weights)
        self.dense.apply(self._init_weights)

    def _init_weights(self, module):
        """ Initialize the weights """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=1)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self, inputs, sequence_length_lst):
        packed_input = pack_padded_sequence(inputs, sequence_length_lst.cpu(), batch_first=True, enforce_sorted=False)

        output_packed, (h, c) = self.lstm(packed_input)

        padded_output, lengths = pad_packed_sequence(output_packed, batch_first=True)

        # classifier
        x = self.dropout(padded_output)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        
        return x

# Loss Function

In [ ]:
# packed_input = pack_padded_sequence(x, seq_len, batch_first=True, enforce_sorted = False)

In [ ]:
# # packed_input
# packed_input.data.shape

In [ ]:
# test_lstm = nn.LSTM(384, 384, batch_first=True)
# # test_lstm.to(device)

In [ ]:
# # Batch X length X Dimension
# output_packed, (h, c) = test_lstm(packed_input)

In [ ]:
# # Batch X length X Dimension 
# padded_output, lengths = pad_packed_sequence(output_packed, batch_first=True)

In [ ]:
# print(padded_output.shape, lengths.shape)

In [ ]:
# drop_out_rate = 0.2
# dropout = nn.Dropout(drop_out_rate)
# dense = nn.Linear(384, 384)
# out_proj = nn.Linear(384, 2)


In [ ]:
# x = dropout(padded_output)
# x = dense(x)
# x = torch.tanh(x)
# x = dropout(x)
# x = out_proj(x)

In [ ]:
# print(x.shape)
# print(y.shape)

In [ ]:
def criterion_2(yhat, y, ignore_index=-100):
    # Merge batch and sequence dimension into one
    y = y.view(-1)
    yhat = F.log_softmax(yhat, dim=-1).view(-1, yhat.shape[-1])
    
    # Get mask for target values != padding index
    nonpad_mask = y != ignore_index
    
    # Slice out non-pad values
    y_nonpad = y[nonpad_mask]
    yhat_nonpad = yhat[nonpad_mask]
    
    # Get the model's probability of the correct class
    prob = yhat_nonpad[range(yhat_nonpad.shape[0]), y_nonpad]
    
    # Calculate loss
    loss = (- prob).mean()
    return loss

In [ ]:
def criterion(logits, labels, ignore_index=-100):
    # Merge batch and sequence dimension into one
    # (Batch, Length, label) -> (Batch x Length, label)
    labels = labels.view(-1)
    
    # (Batch, Length, label) -> (Batch x Length, label) 
    logits = F.log_softmax(logits, dim=-1).view(-1, logits.shape[-1])
    
    # Get mask for target values != padding index
    nonpad_mask = labels != ignore_index
    
    # Slice out non-pad values
    labels_nonpad = labels[nonpad_mask]
    logits_nonpad = logits[nonpad_mask]
    
    # Get the model's probability of the correct class
    prob = logits_nonpad[range(logits_nonpad.shape[0]), labels_nonpad]
    
    # Calculate loss
    loss = (- prob).mean()
    return loss

In [ ]:
# print(x.shape)
# print(y.shape)

### 1. method

In [ ]:
# loss_1 = criterion(x, y)
# print(loss_1)

### 2. method

In [ ]:
# loss = criterion_2(x, y)
# print(loss)

### 3. method

In [ ]:
# criterion = nn.CrossEntropyLoss(ignore_index=-100)
# loss = criterion(x.view(-1, 2), y.view(-1))
# print(loss)

# Train

In [ ]:
model = SentenceLSTM(num_layers=2, embedding_size=384, num_labels=2, drop_out_rate=0.2)

In [ ]:
# for step, batch in enumerate(train_dataloader):
#     outputs = model(batch['inputs'], batch['sequence_len'])
#     break

In [ ]:
# print(outputs.shape)
# print(batch['labels'].shape)

In [ ]:
# print(outputs.is_cuda)
# print(batch['labels'].is_cuda)

In [ ]:
# loss_1 = criterion(outputs, batch['labels'])
# print(loss_1)

In [ ]:
# batch['labels'].view(-1).shape

In [ ]:
# outputs.view(-1, outputs.shape[-1]).shape

In [ ]:
# criterion = nn.CrossEntropyLoss(ignore_index=-100)
# loss = criterion(outputs.view(-1, outputs.shape[-1]), batch['labels'].view(-1))

In [ ]:
# loss

In [ ]:
# outputs.shape

# Eval

In [ ]:
model = SentenceLSTM(num_layers=2, embedding_size=384, num_labels=2, drop_out_rate=0.2)

In [ ]:
metric_acc = evaluate.load("accuracy")
metric_pre = evaluate.load('precision')
metric_re = evaluate.load('recall')
metric_f1 = evaluate.load('f1')

In [ ]:
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed

In [ ]:
accelerator = Accelerator()

In [ ]:
model = SentenceLSTM(num_layers=2, embedding_size=384, num_labels=2, drop_out_rate=0.2)

In [ ]:
eval_file_path = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/sequence_exclude_no_answer_exclude_indecisve/sequence_exclude_no_answer_exclude_indecisve_ctx100id_split_dev_1.json'

In [ ]:
eval_data = utils.open_json(eval_file_path)

In [ ]:
eval_data = utils.prepare_sequential_data(eval_data)

In [ ]:
eval_dataset = SentenceClassificationDataset(seq_train_data, embedding_model)

In [ ]:
eval_dataloader = DataLoader(eval_dataset,
                              shuffle = False,
                              collate_fn=custom_collate,
                              batch_size=128,
)

In [ ]:
# moving from cpu to gpu
model, eval_dataloader = accelerator.prepare(
        model, eval_dataloader,
    )

In [ ]:
from tqdm.auto import tqdm

In [ ]:
eval_progress_bar = tqdm(range(len(eval_dataloader)), disable=not accelerator.is_local_main_process)

In [ ]:
eval_loss = 0
model.eval()
samples_seen = 0
prediction_lst = []
reference_lst = []

for step, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        logits = model(batch['inputs'], batch['sequence_len'])
        criterion = torch.nn.CrossEntropyLoss(ignore_index=-100).cuda()
        eval_loss = criterion(logits.view(-1, 2), batch['labels'].view(-1))
    
    eval_loss += eval_loss.detach().float()
    
    predictions = logits.argmax(dim=-1)
    references = batch['labels']
    
    # Get mask for target values != padding index
    nonpad_mask = references != -100
    
    # Slice out non-pad values
    references = references[nonpad_mask]
    predictions = predictions[nonpad_mask]
    
    predictions, references = accelerator.gather((predictions, references))
    # If we are in a multiprocess environment, the last batch has duplicates
    if accelerator.num_processes > 1:
        if step == len(eval_dataloader) - 1:
            predictions = predictions[: len(eval_dataloader.dataset) - samples_seen]
            references = references[: len(eval_dataloader.dataset) - samples_seen]
        else:
            samples_seen += references.shape[0]
            
    metric_acc.add_batch(
            predictions=predictions,
            references=references,
        )
    metric_pre.add_batch(
        predictions=predictions,
        references=references,
    )
    metric_re.add_batch(
        predictions=predictions,
        references=references,
    )
    metric_f1.add_batch(
        predictions=predictions,
        references=references,
    )
    eval_progress_bar.update(1)
    prediction_lst.extend(predictions.detach().cpu().tolist())
    reference_lst.extend(references.detach().cpu().tolist())
    
    

In [ ]:
eval_metric = metric_acc.compute()
eval_metric_pre = metric_pre.compute()
eval_metric_re = metric_re.compute()
eval_metric_f1 = metric_f1.compute()

In [ ]:
print(eval_metric)
print(eval_metric_pre)
print(eval_metric_re)
print(eval_metric_f1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
len(prediction_lst)

In [ ]:
len(reference_lst)

In [ ]:
import pandas as pd

In [ ]:
## Extra
prediction_np = np.array(prediction_lst)
reference_np = np.array(reference_lst)
y_actu = pd.Series(reference_np, name='Actual')
y_pred = pd.Series(prediction_np, name='Predicted')

reversey_pred = y_pred.map(lambda x: 0 if x == 1 else 1)
reversey_actu = y_actu.map(lambda x: 0 if x == 1 else 1)
rev_accuracy = accuracy_score(reversey_actu, reversey_pred)
rev_precision = precision_score(reversey_actu, reversey_pred)
rev_recall = recall_score(reversey_actu, reversey_pred)
rev_f1 = f1_score(reversey_actu, reversey_pred)

# logger.info(f"rev Evaluation at Epoch : {epoch} Total Step : {steps}")
# logger.info(f"rev_Accuracy : {rev_accuracy} rev_Precision : {rev_precision}")
# logger.info(f"rev_Recall : {rev_recall} rev_F1 : {rev_f1}")
# logger.info(f"Epoch : {epoch} Step : {steps}")
# logger.info(f"Eval_loss : {eval_loss.item() / len(eval_dataloader)}")

result_rev_log = {
    "eval_rev_accuracy": rev_accuracy,
    "eval_rev_precision": rev_precision,
    "eval_rev_recall": rev_recall,
    "eval_rev_f1": rev_f1,
    "eval_loss": eval_loss.item() / len(eval_dataloader),
#     "epoch": epoch,
#     "step": steps,
}

In [ ]:
pprint(result_rev_log)

In [ ]:
print(predictions.shape)
print(references.shape)

In [ ]:
references = references.view(-1)
predictions = predictions.view(-1)

In [ ]:
print(references.shape)
print(predictions.shape)

In [ ]:
nonpad_mask = references != -100

In [ ]:
print(nonpad_mask.shape)

In [ ]:
references = references[nonpad_mask]
predictions = predictions[nonpad_mask]

In [ ]:
print(references.shape)
print(predictions.shape)

In [ ]:
torch.sum(batch['sequence_len'])